In [1]:
# from models.py
from django.db.models import Model
from djongo import models as djongo_models
from django.conf import settings

ModuleNotFoundError: No module named 'django'

In [620]:
# from views.py
#from django.contrib import messages as gui_messages
#from django.contrib.messages import get_messages

from upload.models import (
    DatamodelSource,
    DatamodelUnit,
    DatamodelAttribute,
    DatamodelCode,
    DatamodelAttributeMapping,
    DatamodelCodeMapping,
    DatamodelCalculation,
    UserFile,
)

from pymongo.errors import BulkWriteError

In [196]:
# simple libraries
import calendar
import csv
import re
from datetime import date, datetime
from collections import Counter

# custom import
from upload.BulkCreateManager import BulkCreateManager

In [197]:
# PyExcel
from pyexcel_io import save_data
from pyexcel import get_book
from pyexcel import get_sheet
from pyexcel_io.constants import DB_DJANGO
from pyexcel_io.database.common import DjangoModelImporter, DjangoModelImportAdapter


In [198]:
# UCUM
from pyucum.ucum import *
import urllib
import xml.etree.ElementTree as ET

In [199]:
# global variables
true_replacements = [True, "True", "TRUE", "true"]
false_replacements = [False, "False", "FALSE", "false"]
date_range_global = ["1875-01-01", datetime.today().strftime('%Y-%m-%d')]
ucum_api_url = "http://ucum.nlm.nih.gov/ucum-service/v1"
colname_map = {'Sources':            ["Abbreviation", "Source", "PID_colname",
                                      "SITE_colname", "TIMESTAMP_colname", "Header_offset"],
               'Units':              ["Unit", "UCUM", "Description"],
               'Attributes':         ["Active", "Topic", "Topic_Description", "Umbrella", "Umbrella_Description",
                                      "Attribute", "Attribute_Description", "Attribute_Tooltip",
                                      "Datatype", "Domain", "Unit"],
               'Codes':              ["Active", "Code", "Code_Description", "Key", "Value"],
               'Attribute_Mappings': ["Active", "Source", "Source_Attribute", "Target_Attribute", "Transformation"],
               'Code_Mappings':      ["Active", "Code_Mapping", "Source_Value", "Source_Value_Description",
                                      "Target_Equivalent", "Remarks"],
               'Calculations':       ["Active", "Workbench", "Source", "Attribute", "Function", "Remarks"]
               }

main_msg_queue = []


In [200]:
def toe_error(msg, queue=None):
    if queue is not None:
        queue.append("ERROR: " + msg)
        return queue
    raise Exception(msg)

def toe_warning(msg, queue=None):
    if queue is not None:
        queue.append("WARNING: " + msg)
        return queue
    print("WARNING: " + msg)

def toe_info(msg, queue=None):
    if queue is not None:
        queue.append("INFO: " + msg)
        return queue
    print("INFO: " + msg)

def throw_or_enqueue(what, msg, queue=None):
    if what == "error":
        toe_error(msg, queue)
    elif what == "warning":
        toe_warning(msg, queue)
    elif what == "info":
        toe_info(msg, queue)
    else:
        raise Warning("Could not assign '" + msg + "' to a valid message class!")

        
# helper function for handling exceptions in list comparisons
def catch(func, handle=lambda e: e, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return handle(e)
    

In [201]:
def drop_tables_data():
    DataPoints.objects.all().delete()

def drop_tables_mapping():
    drop_tables_data()
    DatamodelCalculation.objects.all().delete()
    DatamodelAttributeMapping.objects.all().delete()
    DatamodelCodeMapping.objects.all().delete()

def drop_tables_core():
    drop_tables_mapping()
    DatamodelAttribute.objects.all().delete()
    DatamodelCode.objects.all().delete()
    DatamodelUnit.objects.all().delete()
    DatamodelSource.objects.all().delete()


In [202]:
def get_dependency_levels_core():
    model_dependency_levels = [[DatamodelSource,
                                DatamodelUnit,
                                DatamodelCode,
                                ],
                               [DatamodelAttribute],
                              ]
    return model_dependency_levels

def get_dependency_levels_mapping():
    model_dependency_levels = [[DatamodelCodeMapping],
                               [DatamodelAttributeMapping],
                               [DatamodelCalculation],
                              ]
    return model_dependency_levels


In [203]:
def get_header_indices(act_colnames, exp_colnames):
    indices = {n: catch(lambda: act_colnames.index(n)) for n in exp_colnames}
    #indices = {n: catch(lambda: act_colnames.index(exp_colnames[n])) for n in exp_colnames}
    misses = [re.findall('\'([^\']*)\'', miss.args[0])[0] for miss in indices.values() if
              miss.__class__ is ValueError]
    if misses:
        msg = "Could not find following column headers (format corrupt?):\n" + "\n".join(misses)
        throw_or_enqueue("error", msg, main_msg_queue)
        return False
    return indices


In [204]:
def get_bool_fields(model):
    return [f.name for f in model._meta.fields if f.__class__.__name__ == 'BooleanField']


def get_char_fields(model):
    return [f.name for f in model._meta.fields if f.__class__.__name__ == 'CharField']


def get_relations(model):
    return [f for f in model._meta.fields if f.is_relation]


In [358]:
# variables usually acquired from the web interface (user inputs)
filepath = "C:/Users/sschaffner/sciebo/IDSNneu/DataModel/Zusammenführung/DM20200826.xlsx"
header_line_core = {'Sources': 0,    # offset
                    'Units': 0,
                    'Attributes': 0,
                    'Codes': 0,
                    }
header_line_mapping = {'Attribute_Mappings': 0,    # offset
                       'Code_Mappings': 0,
                       'Calculations': 0,
                       }
model2sheet_core = {DatamodelSource: 'Sources',
                    DatamodelUnit: 'Units',
                    DatamodelCode: 'Codes',
                    DatamodelAttribute: 'Attributes',
                    }
model2sheet_mapping = {DatamodelAttributeMapping: 'Attribute_Mappings',
                       DatamodelCodeMapping: 'Code_Mappings',
                       DatamodelCalculation: 'Calculations',
                       }

write_modes = [("new", "Drop complete datamodel before loading file"),
               ("add", "Leave existing entries untouched, append your new ones"),
               ]
write_mode = write_modes[0][0]    # "add" - "new" waere eine Einstellung *nur* fuer uns


In [206]:
# checks on numerical values, mainly dedicated to def check_domain

def represents_int(s):
    try:
        int(s)
        return True
    except ValueError:
        return False


def lossless_float2int(numstring):
    if numstring.count(".") == 1:
        integer, decimal = numstring.split(".")
        if represents_int(integer):
            if not decimal or (represents_int(decimal) and int(decimal) == 0):
                msg = "Converted '" + numstring + "' to integer '" + integer + "' (lossless)"
                throw_or_enqueue("warning", msg, main_msg_queue)
                return integer
    msg = "Could not convert float value to integer number: " + numstring
    throw_or_enqueue("error", msg, main_msg_queue)
    return False


def check_int_array_and_sanitize(str_int):
    int_range_pattern = re.compile("(^(-?\d)?\d*:(-?\d)?\d*$)|(^(-?\d)?\d*$)")
    elements = str_int.split(",")
    misfits = [i for i in range(len(elements)) if not int_range_pattern.match(elements[i])]
    for m in misfits:
        x = elements[m].split(":")
        if len(x) > 2:
            return False
        try:
            elements[m] = ":".join([str(f) for f in [lossless_float2int(x[i]) if x[i] else "" for i in range(len(x))]])
        except ValueError:
            return False
    return ",".join(elements)


def check_float_array_and_sanitize(str_float):
    float_range_pattern = re.compile("(^(-\d+)|(\d+)\.\d+$)|(^((-\d+)|(\d+)\.\d+)?:((-\d+)|(\d+)\.\d+)?$)")
    elements = str_float.split(",")
    misfits = [i for i in range(len(elements)) if not float_range_pattern.match(elements[i])]
    for m in misfits:
        x = elements[m].split(":")
        if len(x) > 2:
            return False
        try:
            elements[m] = ":".join([str(f) for f in [float(x[i]) if x[i] else "" for i in range(len(x))]])
        except ValueError:
            return False
    return ",".join(elements)


In [207]:
# checks on date values

def sanitize_date(d):
    # remove time stamp
    old = d
    d = re.compile("[0-9][0-9]:[0-9][0-9]:[0-9][0-9]").sub("", d)
    if not d == old:
        msg = "Removed timestamp from date."
        throw_or_enqueue("warning", msg, main_msg_queue)

    # remove leading/trailing spaces
    old = d
    d = d.strip()
    if not len(old) == len(d):
        msg = "Removed leading/trailing whitespaces."
        throw_or_enqueue("warning", msg, main_msg_queue)

    # if dot "." as separator: apply dash "-"
    old = d
    d = d.replace(".", "-")
    if not d == old:
        throw_or_enqueue("info", "Replaced '.' with correct separators '-'.", main_msg_queue)

    # remove leading/trailing separators
    old = d
    d = d.strip("-")
    if not len(old) == len(d):
        msg = "Removed dangling separators."
        throw_or_enqueue("warning", msg, main_msg_queue)

    # for incomplete dates: add "00" blocks as placeholder
    dash_cnt = d.count("-")
    if dash_cnt == 1:
        # messages.warning("Incomplete date - added day field '00'")
        if re.compile("^[0-9][0-9][0-9][0-9]-[0-9][0-9]").match(d):    # trailing month in template (preferred)
            msg = "Incomplete date - added day field '00'"
            throw_or_enqueue("warning", msg, main_msg_queue)
            return d + "-00"
        elif re.compile("^[0-9][0-9]-[0-9][0-9][0-9][0-9]").match(d):  # leading month in template (to be corrected later)
            msg = "Incomplete date - added day field '00'"
            throw_or_enqueue("warning", msg, main_msg_queue)
            return "00-" + d
        else:
            msg = "Invalid partial date pattern: '" + d + "'"
            throw_or_enqueue("error", msg, main_msg_queue)
            return False
    elif dash_cnt == 0:
        msg = "Incomplete date - added day and month fields '00'"
        throw_or_enqueue("warning", msg, main_msg_queue)
        return d + "-00-00"
    elif dash_cnt != 2:
        msg = "Inadequate number of separators (" + str(dash_cnt) + ") found!"
        throw_or_enqueue("error", msg, main_msg_queue)
        return False
    else:
        return d


def reverse_date(d):
    return "-".join(reversed(d.split("-")))


def check_date_pattern(d, sanitize):
    pat_msg = "'YYYY-MM-DD' required, with '00' legal for 'DD' and 'MM'"
    pattern = re.compile("^[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]$")
    if pattern.match(d):
        return d
    elif not sanitize:
        return False

    rev_d = reverse_date(d)
    if pattern.match(rev_d):
        msg = "Reversed date sequence - " + pat_msg
        throw_or_enqueue("warning", msg, main_msg_queue)
        return rev_d

    if re.compile("^[0-9][0-9]-").match(d):
        msg = "No valid date pattern! (presumably two-digit YEAR expression - " + pat_msg + ")"
        throw_or_enqueue("warning", msg, main_msg_queue)
        return False
    elif re.compile("^[0-9][0-9][0-9][0-9]-[0-9]-[0-9][0-9]$").match(d):
        msg = "No valid date pattern! (presumably single-digit MONTH expression - " + pat_msg + ")"
        if not sanitize:
            throw_or_enqueue("error", msg, main_msg_queue)
            return False
    elif re.compile("^[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9]$").match(d):
        msg = "No valid date pattern! (presumably single-digit DAY expression - " + pat_msg + ")"
        if not sanitize:
            throw_or_enqueue("error", msg, main_msg_queue)
            return False
    elif re.compile("^[0-9][0-9][0-9][0-9]-[0-9]-[0-9]$").match(d):
        msg = "No valid date pattern! (presumably single-digit MONTH and DAY expression - " + pat_msg + ")"
        if not sanitize:
            throw_or_enqueue("error", msg, main_msg_queue)
            return False
    
    throw_or_enqueue("warning", msg, main_msg_queue)
    year, month, day = d.split("-")
    if len(month) == 1:
        month = "0" + month
    if len(day) == 1:
        day = "0" + day

    # recursion (effectively single iteration)
    d = check_date_pattern(year + "-" + month + "-" + day, sanitize=False)

    if d:
        msg = "Sanitized invalid date format."
        throw_or_enqueue("warning", msg, main_msg_queue)
        return d
    msg = "No valid date pattern! (attempts to sanitize failed)"
    throw_or_enqueue("error", msg, main_msg_queue)
    return False


def calculate_date_object(d):
    year, month, day = d.split("-")
    return date(int(year), int(month), int(day))


def fit_00_date_vs_range(d, date_range):
    year, month, day = d.split("-")  # pattern YYYY-MM-DD ensured here

    msg = "Date '" + d + "' out of accepted range! ('" + "' to '".join(date_range) + "')"
    if int(month) * int(day):  # both values are unequal zero - fully python-valid date format
        if not calculate_date_object(date_range[0]) <= calculate_date_object(d) <= calculate_date_object(date_range[1]):
            throw_or_enqueue("error", msg, main_msg_queue)
            return False
        else:
            return d

    # day and/or month are zero'd - check year match first
    if not int(date_range[0][0:4]) <= int(year) <= int(date_range[1][0:4]):
        throw_or_enqueue("error", msg, main_msg_queue)
        return False

    if not int(month):
        month_min = "12"
        month_max = "01"
    else:
        month_min = month_max = month

    if not int(day):
        date_min = year + "-" + month_min + "-" + str(calendar.monthrange(int(year), int(month_max))[1]).zfill(2)
        date_max = year + "-" + month_max + "-01"
    else:
        date_min = year + "-" + month_min + "-" + day
        date_max = year + "-" + month_max + "-" + day

    if calculate_date_object(date_range[0]) <= calculate_date_object(date_min):
        if calculate_date_object(date_max) <= calculate_date_object(date_range[1]):
            return d

    throw_or_enqueue("error", msg, main_msg_queue)
    return False


# date: central access
def check_date(d, sanitize=False, date_range=date_range_global):
    # expected pattern: YYYY-MM-DD

    # sanitize?
    if sanitize:
        d = sanitize_date(d)
        if not d: return False

    # test character set
    if not re.compile("^(\d+(?:-\d+)*)$").match(d):
        msg = "Illegal characters in date detected."
        throw_or_enqueue("error", msg, main_msg_queue)
        return False

    # check pattern
    d = check_date_pattern(d, sanitize)
    if not d: return False

    # check plausible ranges
    year, month, day = d.split("-")
    if not 0 <= int(day) <= 31:
        msg = "Invalid day value (" + str(day) + ")!"
        throw_or_enqueue("error", msg, main_msg_queue)
        d = False
    if not 0 <= int(month) <= 12:
        msg = "Invalid month value (" + str(month) + ")!"
        throw_or_enqueue("error", msg, main_msg_queue)
        d = False

    if d: d = fit_00_date_vs_range(d, date_range)

    return d


def check_date_array_and_sanitize(str_date):
    elements = [r.strip() for r in str_date.split(",")]
    # corrected = []
    error_flag = False
    for i in range(len(elements)):
        x = elements[i].split(":")
        if len(x) > 2:
            return False
        try:
            tmp = [str(f) for f in [check_date(x[j], date_range=date_range_global, sanitize=True) if x[j] else "" for j in range(len(x))]]
            if calculate_date_object(tmp[0]) > calculate_date_object(tmp[1]):
                msg = "'" + str_date + "' => 'from' date greater than 'to' date in date range!"
                throw_or_enqueue("error", msg, main_msg_queue)
                error_flag = True
            elements[i] = ":".join(tmp)
        except ValueError:
            return False
    if error_flag:
        return False
    return ",".join(elements)


In [208]:
def prepare_sheet(sheet):

    # Check header (sequence of expected columns might differ)
    indices = get_header_indices(sheet.colnames, colname_map[sheet_name])
    if not indices:
        msg = "Corrupt header line for '" + sheet_name + "' - please check log for details."
        throw_or_enqueue("error", msg, main_msg_queue)
        return False

    # Remove unused COLUMNS
    delete_columns = [i for i in range(sheet.number_of_columns()) if not sheet.colnames[i] in indices]
    column_mapper = [i for i in range(sheet.number_of_columns()) if i not in delete_columns]
    sheet.delete_columns(delete_columns)

    # Index ROWS with Active = FALSE, if given
    inactive_rows = []
    if "Active" in sheet.colnames:
        active_col = sheet.column["Active"]
        inactive_rows = [i for i in range(sheet.number_of_rows()) if active_col[i] in [0] + false_replacements]

    # Index blank ROWS
    blank_rows = [i for i in range(sheet.number_of_rows()) if blank_row(i, sheet.row[i])]

    # Remove blank and inactive ROWS from sheet
    # ToDo: report inactive lines as infos, blank lines as warnings
    delete_rows = inactive_rows + blank_rows  # no overlap of lists possible...
    row_mapper = [i for i in range(sheet.number_of_rows()) if i not in delete_rows]
    sheet.delete_rows(delete_rows)

    return sheet, row_mapper, column_mapper


In [209]:
# check column for compliance with model's respective field
def check_regular_field_compliance(colname, sheet, model, fault_collector):
    col = sheet.column[colname]
    field = model._meta.get_field(colname)
    bool_fields = get_bool_fields(model)
    char_fields = get_char_fields(model)

    ## general: blank AND blank allowed?
    if not (field.blank and field.null) and field._get_default() is None:
        fault_collector[colname]["blank"] = [i for i in range(len(col)) if col[i] == ""]

    ## boolean fields: convertable?
    if colname in bool_fields:
        msg_base = "Column '" + colname + "' expected to be either TRUE or FALSE - "
        colset = set([str(y) for y in col])
        diff = colset.difference(["0", "1"])
        if diff:  # contents not read from regular Excel bool cells - try adequate values
            throw_or_enqueue("warning", msg_base + "found '" + "', '".join(diff) + "'", main_msg_queue)
            replaced_true = [1 if x in true_replacements else x for x in col]
            replaced_false = [0 if x in false_replacements else x for x in replaced_true]
            diff_again = set(replaced_false).difference(["0", "1"])
            if diff_again:  # replacement failed - refuse
                throw_or_enqueue("error", msg_base + "could not sanitize value(s) '" + "', '".join(diff) + "'", main_msg_queue)
                fault_collector[colname]["bools"] = [i for i in range(len(replaced_false)) if
                                                           replaced_false[i] in diff_again]
            else:  # replacement successful - exchange column
                sheet.column[colname] = replaced_false

    ## choices fields: in scope?
    choices = field.choices
    if choices:
        fault_collector[colname]["choices"] = [i for i in range(len(col)) if
                                                     col[i] not in [t[0] for t in choices]]
        if fault_collector[colname]["choices"]:
            fields = "\n  ".join([entry[0] + ": " + entry[1] for entry in field.choices])
            throw_or_enqueue("error", "Values for column '" + colname + "' exceed range of allowed choices:\n" + fields, main_msg_queue)
            
    ## character fields: length?
    if colname in char_fields:
        flag = False
        for i in range(len(col)):
            if col[i].__class__.__name__ != "str":
                col[i] = str(col[i])
                flag = True
        if flag:
            sheet.column[colname] = col
        fault_collector[colname]["length"] = [i for i in range(len(col)) if len(col[i]) > field.max_length]
        if fault_collector[colname]["length"]:
            msg = "Values for column '" + colname + "' exceed maximum string length of " + field.max_length
            throw_or_enqueue("error", msg, main_msg_queue)


In [210]:
# check column for uniqueness, if required by model's respective field
def check_uniqueness_for_column(colname, sheet, model, fault_collector):
    if colname == model._meta.pk.name or model._meta.get_field(colname)._unique:
        # check duplicates locally in column
        col = sheet.column[colname]
        c = Counter(col)
        duplicates = [item for item in c.keys() if c[item] > 1]
        fault_collector[colname]["duplicate"] = [i for i in range(len(col)) if col[i] in duplicates]
        if fault_collector[colname]["duplicate"]:
            msg = "Found duplicate entries in column '" + colname + "' (unique required)"
            throw_or_enqueue("error", msg, main_msg_queue)
        # check versus existing DB objects
        existing = model.objects.values_list(colname, flat=True)
        fault_collector[colname]["assigned"] = [i for i in range(len(col)) if col[i] in existing]
        if fault_collector[colname]["assigned"]:
            msg = "Found entries in column '" + colname + "' already assigned in database (unique required)."
            throw_or_enqueue("error", msg, main_msg_queue)


In [343]:
# Tests, whether declarations of (type-dependent) domains are correct
def check_domain(colname, sheet, codes, fault_collector):
    # get rid off useless blanks and (allowed) brackets
    col = [cell.strip(" []") for cell in sheet.column[colname]]
    if not col == sheet.column[colname]:
        sheet.column[colname] = col
        msg = "Removed useless leading/trailing whitespace characters from column '" + colname + "'"
        throw_or_enqueue("warning", msg, main_msg_queue)
    datatypes = sheet.column["Datatype"]
    fault_collector[colname]["domain"] = []
    db_codes = list(DatamodelCode.objects.values_list("Code", flat=True))
    all_codes = db_codes + codes

    for i in range(len(col)):
        if not len(col[i]):
            continue  # Domain is optional... if empty: skip

        # integer tests
        if datatypes[i] in ["int", "array(int)"]:
            corr = check_int_array_and_sanitize(col[i])
            if corr:
                if not col[i] == corr:
                    msg = "Found float values or whitespace chars in domain declaration of an integer-type variable."
                    throw_or_enqueue("warning", msg, main_msg_queue)
                    col[i] = corr
            else:
                fault_collector[colname]["domain"].append(i)
            continue

        # float tests
        if datatypes[i] in ["float", "array(float)"]:
            corr = check_float_array_and_sanitize(col[i])
            if corr:
                if not col[i] == corr:
                    msg = "Found integer values or whitespace chars in domain declaration of a float-type variable: '" + col[i] + "'"
                    throw_or_enqueue("warning", msg, main_msg_queue)
                    col[i] = corr
            else:
                fault_collector[colname]["domain"].append(i)
            continue

        # code tests
        if datatypes[i] in ["code", "array(code)"]:
            corr = [r.strip() for r in col[i].split(",")]
            if not ",".join(corr) == col[i]:
                msg = "Removed unnecessary whitespaces in codes list."
                throw_or_enqueue("warning", msg, main_msg_queue)
                col[i] = corr
            misfits = [r for r in col[i].split(",") if r not in all_codes]
            if misfits: 
                msg = "Found unknown codes: " + ", ".join(misfits)
                throw_or_enqueue("error", msg, main_msg_queue)
                fault_collector[colname]["domain"].append(i)
            continue

        # date tests
        if datatypes[i] in ["date", "array(date)"]:
            corr = check_date_array_and_sanitize(col[i])
            if corr:
                if not col[i] == corr:
                    msg = "Found sub-optimal formatting in domain declaration of a date-type variable."
                    throw_or_enqueue("warning", msg, main_msg_queue)
                    col[i] = corr
            else:
                fault_collector[colname]["domain"].append(i)
            continue

    # if datatypes[i] == "string" and not array_float_array_pattern.match(col[i]):
    if not col == sheet.column[colname]:
        sheet.column[colname] = col
        msg = "Sanitized entries in column '" + colname + "'"
        throw_or_enqueue("warning", msg, main_msg_queue)


In [212]:
# get rid of empty sheet rows
def blank_row(row_index, row):
    result = [element for element in row if element != '']
    return len(result) == 0


# helper; determines whether issues have been recorded
def get_error_flag(fault_collector, colname):
    if [error for error in fault_collector[colname].keys() if fault_collector[colname][error]]:
        return True
    else:
        return False

In [614]:
## check foreign key columns: string referencing correct (existing) key?
def check_foreign_keys(colname, sheet, model, local_fault_collector, data_collector):
    relations = get_relations(model)
    if colname in [r.name for r in relations]:
        # database entries: simple query on related model instances
        pos = [r.name for r in relations].index(colname)
        distmodel = relations[pos].related_model
        distfield = relations[pos].to_fields[0]
        db_keys = list(distmodel.objects.values_list(distfield, flat=True))

        # sheet entries (for dependency levels > 1)
        # extract from data collector; browse previously stored, lower dependency level sheets
        sheet_keys = []
        distmodel_name = distmodel._meta.model.__name__.lower()
        if distmodel_name in data_collector:
            colpos = data_collector[distmodel_name].colnames.index(distfield)
            sheet_keys.extend([row[colpos] for row in data_collector[distmodel_name]])
        
        col = sheet.column[colname]
        all_keys = set(db_keys + sheet_keys)
        #print(db_keys)
        #print(sheet_keys)
        #print(all_keys)
        if not set(col).issubset(all_keys):  # unknown foreign key referenced
            local_fault_collector[colname]["foreignkey"] = [i for i in range(len(col)) if col[i] not in all_keys]
            for i in local_fault_collector[colname]["foreignkey"]:
                throw_or_enqueue("error", "Unknown target attribute referenced: '" + col[i] + "'", main_msg_queue)


In [628]:
# Setup containers for sheet/model
def fill_import_containers(importer, data_collector, model, sheet):
    # generate import adapter from model
    adapter = DjangoModelImportAdapter(model)
    # print(sheet.colnames)
    apter.column_names = sheet.colnames
    
    # add adapter to importer (= queue for current dependency level)
    importer.append(adapter)

    # feed data collector struct with sheet-derived data
    # data_collector[adapter.get_name()] = sheet.get_internal_array()
    data_collector[adapter.get_name()] = sheet
    print(data_collector)


In [314]:
# float
def floatable(x, sanitize=False):
    if sanitize:
        value = x.replace(",", ".")
        if value != x:
            throw_or_enqueue("warning", "Converted floating point symbol in '" + x + "' (German notation ',' found!).", main_msg_queue)
            x = value
    try:
        return float(x)
    except ValueError as e:
        return False


# float
def advanced_string2float(string, refU_obj, verified_UCUM_units={}, value_conversions={}):
    string = consider_unit_and_convert(string, refU_obj, verified_UCUM_units, value_conversions)
    num = floatable(value)
    if not num:
        throw_or_enqueue("error", "Could not convert value to integer number: '" + string, main_msg_queue)
    return num


# code
def check_codekey_matches(values, target_code, sanitize=False):
    target_keys = set(DatamodelCode.objects.filter(Code=target_code).values_list("Key", flat=True))
    if not target_keys:
        throw_or_enqueue("error", "Could not find code system '" + target_code + "'", main_msg_queue)
        return False
    try:
        diff = set(values).difference(target_keys)
    except KeyError:
        diff = True
    if diff:
        if sanitize:
            fail = []
            success = []
            sanitized = []
            for item in list(values):
                try_hard = str(advanced_string2int(item))
                if try_hard in target_keys:
                    success.append(try_hard)
                    if item != try_hard:
                        sanitized.append(item)
                else:
                    fail.append(item)
            if fail:
                msg = "Found key(s) '" + ", ".join(fail) + "' being incompatible to code system '"\
                      + target_code + "' (not in [" + ", ".join(sorted(target_keys)) + "])"
                throw_or_enqueue("error", msg, main_msg_queue)
                return False
            msg = "Key(s) '" + ", ".join(sanitized) + "' had to be sanitized in order to be compatible to code system '"\
                  + target_code + "'"
            throw_or_enqueue("warning", msg, main_msg_queue)
            return values.__class__(success)
        msg = "Found key(s) '" + ", ".join(values) + "' being incompatible to code system '"\
              + target_code + "' (not in [" + ", ".join(sorted(target_keys)) + "])"
        throw_or_enqueue("error", msg, main_msg_queue)
        return False
    return values


# num + unit
def consider_unit_and_convert(var, refU_obj, verified={}, conversions={}):
    value, unit = separate_value_and_unit(var)

    if not value:  # errornous outputs
        return False

    if not unit:  # no unit detected, keep value as is
        return value

    refU = refU_obj.Unit

    # check if reference or not (yes = no action required)
    if unit == refU:
        return value

    if refU_obj.UCUM:  # verify UCUM membership (automated conversion is yet possible for those only!)
        return convert_using_UCUM(value, unit, refU, verified, conversions)
    else:
        throw_or_enqueue("error", "Auto-conversion from '" + unit + "' to '" + refU\
                         + "' unavailable (no UCUM units).)", main_msg_queue)
        return False


# integer
def represents_int(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

    
# integer
def lossless_float2int(numstring):
    if numstring.count(".") == 1:
        integer, decimal = numstring.split(".")
        if represents_int(integer):
            if not decimal or (represents_int(decimal) and int(decimal) == 0):
                throw_or_enqueue("warning", "Converted '" + numstring + "' to integer '" + integer + "' (lossless)", main_msg_queue)
                return integer
    throw_or_enqueue("error", "Could not convert float value to integer number: " + numstring, main_msg_queue)
    return False


# integer
def advanced_string2int(string, refU_obj=None, verified_UCUM_units={}, value_conversions={}):
    if "," in string:
        string = string.replace(",", ".")
        throw_or_enqueue("error", "Converted '" + string + "' to international decimal '.'!", main_msg_queue)

    if "." in string:
        string = lossless_float2int(string)
        if not string:
            throw_or_enqueue("error", "Could not convert value to integer number: " + string, main_msg_queue)
            return False

    if refU_obj and not refU_obj.Unit == "None":
        string = consider_unit_and_convert(string, refU_obj, verified_UCUM_units, value_conversions)
        if not string:
            throw_or_enqueue("error", "Could not convert value to integer number: " + string, main_msg_queue)
            return False

    try:
        return int(string)
    except ValueError:
        return False


# num (both integer and float)
def check_num_domain_fit(var_num, domain, type):
    for subdomain in array_from_string(domain):
        leftfit = rightfit = True
        left, right = array_from_string(subdomain, delim=":", convert=type)
        if left and var_num < left:
            leftfit = False
        if right and var_num > right:
            rightfit = False
        if leftfit and rightfit:
            return var_num
    throw_or_enqueue("error", "'" + str(var_num) + "' exceeds value range of defined domain: " + str(domain), main_msg_queue)
    return False



100.0

In [216]:
# TODO: implement contents... and call in data import's main()
def apply_transformation(var_int, transformation):
    return var_int


In [217]:
# date
def check_date_domain_fit(var_date, domain):
    # domain = two-item array of dates
    if domain[0] and var_date < domain[0]:
        throw_or_enqueue("error", "'" + str(var_date) + "' exceeds date range: < " + str(domain[0]), main_msg_queue)
        return False
    if domain[1] and var_date > domain[1]:
        throw_or_enqueue("error", "'" + str(var_date) + "' exceeds date range: > " + str(domain[1]), main_msg_queue)
        return False
    return True


# array of elements from formatted string
def array_from_string(instring, delim=",", convert="string"):
    #cont = (instring.split("["))[1].split("]")[0]
    if convert == "int":
        return [int(x) for x in instring.split(delim)]
    elif convert == "float":
        return [float(x) for x in instring.split(delim)]
    elif convert == "string":
        return instring.split(delim)
    else:
        return False


# two-item array of elements ('domain' as from/to pairing)
def domain_from_string(instring, generic_from=False, generic_to=False):
    fr, to = array_from_string(instring)
    if not fr and generic_from: fr = generic_from
    if not to and generic_to: to = generic_to
    return fr, to


# num + unit
def separate_value_and_unit(var):
    var = re.sub(r"\s", "", var)
    value = "".join(re.findall(r"[\d.]", var))
    try:
        startpos = var.index(value)
        if startpos == 0:
            unit = var[(len(value)):]
            return value, unit
        else:
            return False, False
    except ValueError as e:
        msg = "No valid number, even considering trailing unit declarations: " + var
        throw_or_enqueue("error", msg, main_msg_queue)
        return False, False


# num + unit
def UCUM_server_reply2dict(reply):
    d = dict()
    for r in reply:
        u, v = r.split(" = ")
        if v == "true":
            d[u] = True
        elif v == "false":
            d[u] = False
        else:
            d[u] = v
    return d


# num + unit
def verify_units2UCUM(actual_unit, reference_unit, buffer_dict={}):
    call = [x for x in [actual_unit, reference_unit] if x not in buffer_dict]
    if not call:
        return {}
    reply = ucumVerify(call, ucum_api_url)
    reply_dict = UCUM_server_reply2dict(reply)
    if not reference_unit in buffer_dict and not reference_unit in reply_dict:
        throw_or_enqueue("error", "Could not resolve reference unit in UCUM!", main_msg_queue)
        return False
    elif actual_unit in buffer_dict and not actual_unit in reply_dict:
        throw_or_enqueue("error", "Could not resolve reference unit in UCUM!", main_msg_queue)
        return False
    else:
        return reply_dict


# num + unit
def generate_conversion_api_urls(actual_unit, reference_unit, value=1):
    collector = {}
    if isinstance(actual_unit, list) or isinstance(reference_unit, list):
        if len(actual_unit) != len(reference_unit):
            return False
        else:
            for i in range(0, len(actual_unit)):
                collector.update(generate_conversion_api_urls(actual_unit[i], reference_unit[i]))
    else:
        url = ucum_api_url + "/ucumtransform"
        request = url + "/" + str(value) + "/from/" + actual_unit + "/to/" + reference_unit
        request.replace("//", "/")
        collector[actual_unit + "=>" + reference_unit] = request
    return collector


# num + unit
def get_responses_from_UCUM(actU, refU, value):
    urls = generate_conversion_api_urls(actU, refU, value)
    responses = {}
    for conv in urls.keys():
        # request = urls['g/l=>g/dL']
        request = urls[conv]
        try:
            with urllib.request.urlopen(request) as res:
                context = ET.fromstring(res.read())
                # print(context)
                for child in context:
                    tmp1 = {}
                    if child.text == None:
                        for element in child:
                            # print(element.tag)
                            tmp1[element.tag] = element.text
                            # print(child, tmp1)
                    elif child.text != None:  # error handling ERROR: unexpected result: Invalid UCUM Transformation Expression
                        # print(child.text)
                        tmp1["ERROR"] = child.text
        except urllib.error.HTTPError as e:  # error handling bad request
            tmp1["ERROR"] = e
        responses[conv] = tmp1
    return responses


# num + unit
def convert_using_UCUM(value, actU, refU, verified={}, conversions={}):
    reply = verify_units2UCUM(actU, refU, verified)
    if reply is False:
        return False
    verified.update(reply)

    # conversion (for UCUM units)
    try:
        if verified[actU] and verified[refU]:
            conv = actU + "=>" + refU
            rev_conv = refU + "=>" + actU
            if conv in conversions:
                # use 'conversions' dict for calculating the VALUE according to the reference UNIT
                value = str(float(value) * conversions[conv])
            elif rev_conv in conversions:
                # reverse available conversion
                value = str(float(value) / conversions[rev_conv])
            else:
                # query UCUM server API for factor
                responses = get_responses_from_UCUM(actU, refU, value)
                value = responses[conv]['ResultQuantity']
                factor = float(responses[conv]['ResultQuantity']) / float(responses[conv]['SourceQuantity'])
                conversions[conv] = factor
            return value
        else:
            msg = "Auto-conversion from '" + actU + "' to '" + refU + "' unavailable (could not verify using UCUM server)."
            throw_or_enqueue("error", msg, main_msg_queue)
    except KeyError as e:
        throw_or_enqueue("error", "An error occured while trying to convert a UCUM unit...", main_msg_queue)

    return False

#convert_using_UCUM("2", "d", "s")   # True
#convert_using_UCUM("2", "x", "s")   # False

In [572]:
# general entry point for deep tests on all available data types
# TODO: test transformations
# TODO: in data main(), transform input values into target space by source (no mapping/source information necessary to hand in here)
def ensure_datatype_and_domain_fit(values, target, transformation=False, date_range=date_range_global,
                                   sanitize=False, verified_UCUM_units={}, value_conversions={}):
    sanitized = False

    for i in range(len(values)):
        var = values[i]
        c = list()

        if target.Datatype in ["int", "array(int)"]:
            for v in array_from_string(var):
                try:
                    var_int = int(v)
                except ValueError:
                    if sanitize:
                        var_int = advanced_string2int(v, target.Unit, verified_UCUM_units, value_conversions)
                        sanitized = True
                    else:
                        throw_or_enqueue("error", "Could not convert value to integer number: " + v, main_msg_queue)
                        var_int = False
                if var_int and transformation:
                    var_int = apply_transformation(var_int, transformation)
                if var_int and target.Domain:
                    var_int = check_num_domain_fit(var_int, target.Domain, "int")
                if var_int:
                    c.append(str(var_int))
                else:
                    c.append(var_int)

        elif target.Datatype in ["float", "array(float)"]:
            for v in array_from_string(var):
                try:
                    var_float = float(v)
                    if not str(var_float) == v:
                        sanitized = True
                        msg = "Found integer number ('" + v + "') where float expected"
                        if not sanitize:
                            var_float = False
                            throw_or_enqueue("error", msg, main_msg_queue)
                        else:
                            throw_or_enqueue("warning", msg, main_msg_queue)
                except ValueError:
                    if sanitize:
                        var_float = advanced_string2float(var, target.Unit, verified_UCUM_units, value_conversions)
                        sanitized = True
                    else:
                        throw_or_enqueue("error", "Could not convert value to float number: " + var, main_msg_queue)
                        var_float = False
                if var_float and transformation:
                    var_float = apply_transformation(var_float, transformation)
                if var_float and target.Domain:
                    var_float = check_num_domain_fit(var_float, target.Domain, "float")
                if var_float:
                    c.append(str(var_float))
                else:
                    c.append(var_float)

        elif target.Datatype in ["date", "array(date)"]:
            if target.Domain:
                ddomain = [datetime.strptime(x, '%Y-%m-%d') if x else False for x in domain_from_string(target.Domain)]
            for v in array_from_string(var):
                v_ = check_date(v, date_range=date_range, sanitize=sanitize)
                if v_:
                    if v != v:
                        throw_or_enqueue("warning", "Had to sanitize date statement: " + v + " => " + v_, main_msg_queue)
                        v = v_
                    if target.Domain and not check_date_domain_fit(datetime.strptime(v, '%Y-%m-%d'), ddomain):
                        v = False
                else:
                    v = False
                if v:
                    c.append(str(v))
                else:
                    c.append(v)

        elif target.Datatype in ["code", "array(code)"]:
            #key_map = {m.Source_Value: m.Target_Equivalent for m in
            #           DatamodelCodeMapping.objects.filter(Code_Mapping=target.Transformation)}
            for v in array_from_string(var):
                v_ = check_codekey_matches(v, target.Domain, sanitize=sanitize)
                if v_:
                    if v != v:
                        throw_or_enqueue("warning", "Had to sanitize code key: " + v + " => " + v_, main_msg_queue)
                        v = v_
                else:
                    v = False
                if v:
                    c.append(str(v))
                else:
                    c.append(v)

        else:
            throw_or_enqueue("error", "Test for data model variable type '" + target.Datatype + "' is not implemented.", main_msg_queue)
            c.append(False)

        if False not in c:
            values[i] = ",".join(c)
        else:
            values[i] = False

    if False in values:
        return False, sanitized
    
    return values, sanitized


In [608]:
def check_formula(t, restrict=[]):      # 'pattern' parameter for development only!
    functions = ["FORMULA", "MEAN", "RANK", "SUM", "DELTA", "DIV", "PROD", "DRANGE", "DATE"]
    operators = ["+", "-", "/", "*", "%", "^"] #, "(", ")"]
    
    left_bracket = False
    right_bracket = False
    #print("==> '" + t + "'")
    r = t.strip()
    if not r == t:
        t = r
        throw_or_enqueue("warning", "Removed leading/trailing whitespaces from formula.", main_msg_queue)
    
    error_flag = False
    try:
        left_bracket = t.index("(")
    except ValueError:
        error_flag = True
        throw_or_enqueue("error", "No opening bracket in formula.", main_msg_queue)
    try:
        right_bracket = t.rindex(")")
    except ValueError:
        error_flag = True
        throw_or_enqueue("error", "No closing bracket in formula.", main_msg_queue)
    
    if not right_bracket == len(t) - 1:
        error_flag = True
        throw_or_enqueue("error", "Formula declaration does not end with closing bracket.", main_msg_queue)
    function = t[:left_bracket]
    if not function in functions:
        error_flag = True
        if not function:
            msg = "Formula declaration does not start with any function call."
        else:
            msg = "Formula declaration does not start with any known function call - found '" + function + "'..."
        throw_or_enqueue("error", msg, main_msg_queue)
    
    if error_flag:
        msg = "Could not parse a valid formula following scheme 'FUNCTION(parameters | math expr.)'', with FUNCTION out of " \
               + "['" + "', '".join(functions) + "']" + "."
        throw_or_enqueue("error", msg, main_msg_queue)
        return False
    
    inner = t[left_bracket+1:right_bracket]
    if function == "FORMULA":
        delim = " "
        pattern = " (?![^()]*\))"
    else:
        delim = ", "
        pattern = "[,](?![^()]*\))"
    elements = [x.strip() for x in re.split(r''+pattern, inner) if x]
    
    refactored = []
    for e in elements:
        c = []
        for ci in range(len(e)):
            if e[ci] in operators:
                c.append(ci)
        if c:
            first = e[:c[0]].strip()
            if first:
                refactored.append(first)
            for j in range(len(c)-1):
                refactored.append(e[c[j]].strip())
                refactored.append(e[c[j]+1:c[j+1]].strip())
            refactored.append(e[c[-1]].strip())
            last = e[c[-1]+1:].strip()
            if last:
                refactored.append(last)
        else:
            refactored.append(e)
    
    #print(refactored)
    
    misfits = []
    for ri in range(len(refactored)):
        r = refactored[ri]
        if not r:
            continue
        if r in operators:
            continue
        if floatable(r.strip("()")):
            continue
        if "(" in r and r[:r.index("(")] in functions:
            ret = check_formula(r, restrict=restrict)
            if ret:
                if not r == ret:
                    refactored[ri] = ret
                continue
            else:
                misfits.append(r)
                continue
        elif restrict:
            if not r.strip("()") in restrict:
                throw_or_enqueue("error", "Unknown element in formula declaration: '" + r.strip("()") + "'", main_msg_queue)
            else:
                continue
        else:
            value, unit = separate_value_and_unit(r.strip("()"))
            if value:
                if not unit:
                    continue
                else:
                    if convert_using_UCUM(value, unit, "s"):    # check for time tolerance parameter:
                        continue                                #  try conversion to seconds (base unit)
        if restrict:
            misfits.append(r.strip("()"))
    
    if misfits:
        msg = "Issues on formula elements : " + ", ".join(misfits)
        throw_or_enqueue("error", msg, main_msg_queue)
        return False
    
    return t[:left_bracket] + "(" + delim.join([r for r in refactored if r]) + ")"
    

In [607]:
# Formula test setup (ignorieren...)
db_attrs = set(DatamodelAttribute.objects.all().values_list("Attribute", flat=True))
f = []
#f.append(["FORMULA( SUM(adasgcty,adasgctn) *(5+1))", "FORMULA(SUM(adasgcty, adasgctn) * (5 + 1))", False])
f.append(["FORMULA( SUM(SEX, DOB)* 5)", "FORMULA(SUM(SEX, DOB) * 5)", "FORMULA(SUM(SEX, DOB) * 5)"])
#f.append(["MEAN( SEX, DOB)", "MEAN(SEX, DOB)", "MEAN(SEX, DOB)"])
f.append(["FORMULA( (BNT_SUM_Z - 100) / 10) )", "FORMULA((BNT_SUM_Z - 100) / 10))", "FORMULA((BNT_SUM_Z - 100) / 10))"])

for f_ in f:
    ret = check_formula(f_[0])
    if not ret == f_[1]:
        print("!!! " + f_[0] + " ==> " + str(ret))
    else:
        print(f_[0] + " ==> " + str(ret))
    print("-----------------------------------------------------------------------")
    ret = check_formula(f_[0], restrict=db_attrs)
    if not ret == f_[2]:
        print("!!! " + f_[0] + " ==> " + str(ret))
    else:
        print(f_[0] + " ==> " + str(ret))
    print("=======================================================================")
    
for msg in main_msg_queue:
    print(msg)
main_msg_queue = []


['SUM(SEX, DOB)', '*', '5']
['SEX', 'DOB']
FORMULA( SUM(SEX, DOB)* 5) ==> FORMULA(SUM(SEX, DOB) * 5)
-----------------------------------------------------------------------
['SUM(SEX, DOB)', '*', '5']
['SEX', 'DOB']
FORMULA( SUM(SEX, DOB)* 5) ==> FORMULA(SUM(SEX, DOB) * 5)
['(BNT_SUM_Z', '-', '100)', '/', '10)']
FORMULA( (BNT_SUM_Z - 100) / 10) ) ==> FORMULA((BNT_SUM_Z - 100) / 10))
-----------------------------------------------------------------------
['(BNT_SUM_Z', '-', '100)', '/', '10)']
FORMULA( (BNT_SUM_Z - 100) / 10) ) ==> FORMULA((BNT_SUM_Z - 100) / 10))


In [693]:
def check_transformation(colname, sheet, fault_collector, source_code_mappings={}):
    db_code_mappings = list(DatamodelCodeMapping.objects.values_list("Code_Mapping", flat=True))
    db_attrs = set(DatamodelAttribute.objects.all().values_list("Attribute", flat=True))
    source_attrs = set(sheet.column["Source_Attribute"])
    target_column = sheet.column["Target_Attribute"]
    fault_collector[colname]["transformation"] = []
    #formulas = ["FORMULA", "MEAN", "RANK", "SUM", "DELTA", "DIV", "PROD", "DRANGE", "DATE"]
    #operators = ["+", "-", "/", "*", "%", "^"]
    #pattern = ",|\s|\d" + "|[(" + ''.join(operators) + ")]"
    
    col = [cell.strip() for cell in sheet.column[colname]]
    if not col == sheet.column[colname]:
        msg = "Removed useless leading/trailing whitespace characters from column '" + colname + "'"
        throw_or_enqueue("warning", msg, main_msg_queue)
    
    for i in range(len(col)):
        if not col[i]:  # transformation is optional...
            continue
        t = col[i]
        # check, if the referenced code mapping is compatible to target variable's domain by values
        if t in db_code_mappings:
            #print(str(i) + ": " + t + " [CM]")
            cm_targeted_keys = list(set(DatamodelCodeMapping.objects.filter(Code_Mapping=t).values_list("Target_Equivalent", flat=True)))
        elif t in source_code_mappings:
            #print(str(i) + ": " + t + " [cm]")
            cm_targeted_keys = sorted(source_code_mappings[t])
        else:  # no code mapping found... should be a valid formula
            #print(str(i) + ": " + t + " [PF]")
            corr = check_formula(t, restrict=list(db_attrs.union(source_attrs)))
            #print( str(i) + ": " + str(col[i]) + " ==> " + str(corr) )
            if corr:
                col[i] = corr
            else:
                fault_collector[colname]["transformation"].append(i)
            continue
        
        # found a code mapping; value space acquired above
        target_obj = DatamodelAttribute.objects.filter(Attribute=target_column[i])[0]
        cm_targeted_keys_new, sanitized = ensure_datatype_and_domain_fit(cm_targeted_keys, target_obj)
        
        #cm_targeted_keys = check_codekey_matches(cm_targeted_keys, target_obj.Domain)
        if not cm_targeted_keys_new:
            msg = "Could not apply code mapping '" + t + "'; mismatches targeted attribute '" + target_obj.Attribute + "'..."
            throw_or_enqueue("error", msg, main_msg_queue)
            fault_collector[colname]["transformation"].append(i)
        
        sheet.column[colname] = col


In [692]:
def check_function(colname, sheet, fault_collector):
    db_attrs = set(DatamodelAttribute.objects.all().values_list("Attribute", flat=True))
    source_attrs = set(sheet.column["Attribute"])
    #target_column = sheet.column["Target_Attribute"]
    fault_collector[colname]["function"] = []
    #formulas = ["FORMULA", "MEAN", "RANK", "SUM", "DELTA", "DIV", "PROD", "DRANGE", "DATE"]
    #operators = ["+", "-", "/", "*", "%", "^"]
    #pattern = ",|\s|\d" + "|[(" + ''.join(operators) + ")]"
    
    col = [cell.strip() for cell in sheet.column[colname]]
    if not col == sheet.column[colname]:
        msg = "Removed useless leading/trailing whitespace characters from column '" + colname + "'"
        throw_or_enqueue("warning", msg, main_msg_queue)
    
    for i in range(len(col)):
        corr = check_formula(col[i])
        #print( str(i) + ": " + str(col[i]) + " ==> " + str(corr) )
        if corr:
            col[i] = corr
        else:
            fault_collector[colname]["function"].append(i)
        continue
    
    sheet.column[colname] = col


In [224]:
def get_relations(model):
    return [f for f in model._meta.fields if f.is_relation]


def foreign_key_replacement(target_model, target_colname, column, blank_allowed=False):  # toDo: enable reporting
    i = 0
    while i < len(column):
        entry = column[i]
        if not entry:
            if not blank_allowed:
                msg = "Foreign Key field '" + target_colname \
                + "' could not be served (empty source file field in input line " \
                + str(i + 2) + ")!"
                throw_or_enqueue("error", msg, main_msg_queue)
                return False
            column[i] = ''
        else:
            try:
                targetObj = target_model.objects.filter(**{target_colname: entry})[0]
            except IndexError:
                msg = "Foreign Key field '" + target_colname \
                      + "' could not be served (key '" + entry \
                      + "' could not be resolved)!"
                throw_or_enqueue("error", msg, main_msg_queue)
                return False
            column[i] = targetObj
        i += 1
    return column


def setup_foreign_keys_in_sheet(sheet, model):
    relations = get_relations(model)
    i = 0
    for colname in sheet.colnames:  # iterate over sheet-derived names
        try:
            fk_index = [r.name for r in relations].index(colname)
            target_model = relations[fk_index].related_model
            target_column = relations[fk_index].to_fields[0]
            msg = " --> Resolving Foreign Keys..."
            throw_or_enqueue("info", msg, main_msg_queue)
            blank_allowed = relations[fk_index].blank and relations[fk_index].null
            new_col = foreign_key_replacement(target_model, target_column, sheet.column[colname], blank_allowed)
            if not new_col:
                return False
            sheet.column[colname] = new_col
        except ValueError:
            # regular_fields
            pass
        i += 1
    return sheet


In [576]:
def sheet2model_core(sheet, model, data_collector, domain_extension=[]):
    
    error_flag = False
    
    sheet, row_mapper, column_mapper = prepare_sheet(sheet)
    
    # For all remaining data, check model compliance (by columns)
    fault_collector = {}
    msg = "\tWorking on columns:"
    throw_or_enqueue("info", msg, main_msg_queue)
    for colname in colname_map[sheet_name]:
        msg = "\t * " + colname
        throw_or_enqueue("info", msg, main_msg_queue)
        fault_collector[colname] = {}

        check_regular_field_compliance(colname, sheet, model, fault_collector)

        check_uniqueness_for_column(colname, sheet, model, fault_collector)

        if colname == "Domain":
            check_domain(colname, sheet, domain_extension, fault_collector)

        if not error_flag:
            error_flag = get_error_flag(fault_collector, colname)

        if not error_flag:
            check_foreign_keys(colname, sheet, model, 
                               fault_collector, 
                               data_collector)
        
        if not error_flag:
            error_flag = get_error_flag(fault_collector, colname)
    # END OF LOOP over columns of sheet
    
    if not error_flag:  # prepare DB import of data IF no errors occurred yet
        # generate import adapter from model
        adapter = DjangoModelImportAdapter(model)
        adapter.column_names = sheet.colnames
        # feed data collector struct with sheet-derived data
        data_collector[adapter.get_name()] = sheet
        return adapter, fault_collector, row_mapper, column_mapper 
    return False, fault_collector, row_mapper, column_mapper 


In [656]:
def sheet2model_mapping(sheet, model, data_collector, source_code_mappings={}):
    
    error_flag = False
    
    sheet, row_mapper, column_mapper = prepare_sheet(sheet)
    
    # For all remaining data, check model compliance (by columns)
    fault_collector = {}
    msg = "\tWorking on columns:"
    throw_or_enqueue("info", msg, main_msg_queue)
    for colname in colname_map[sheet_name]:
        msg = "\t * " + colname
        throw_or_enqueue("info", msg, main_msg_queue)
        fault_collector[colname] = {}

        check_regular_field_compliance(colname, sheet, model, fault_collector)

        check_uniqueness_for_column(colname, sheet, model, fault_collector)

        if colname == "Transformation":
            check_transformation(colname, sheet, fault_collector,
                                 source_code_mappings=source_code_mappings)
        
        if colname == "Function":
            check_function(colname, sheet, fault_collector)

        if not error_flag:
            error_flag = get_error_flag(fault_collector, colname)

        if not error_flag:
            check_foreign_keys(colname, sheet, model, 
                               fault_collector, 
                               data_collector)
        
        if not error_flag:
            error_flag = get_error_flag(fault_collector, colname)
    # END OF LOOP over columns of sheet
    
    if not error_flag:  # prepare DB import of data IF no errors occurred yet
        # generate import adapter from model
        adapter = DjangoModelImportAdapter(model)
        adapter.column_names = sheet.colnames
        # feed data collector struct with sheet-derived data
        data_collector[adapter.get_name()] = sheet
        return adapter, fault_collector, row_mapper, column_mapper 
    return False, fault_collector, row_mapper, column_mapper 


In [666]:
def importer2database(importer, data_collector):
    error_flag = False
    data_collector_subset = {}  # prepare struct for respective data chunk
    for model_adapter_name in importer._DjangoModelImporter__adapters.keys():
        sheet = data_collector[model_adapter_name]
        model = importer._DjangoModelImporter__adapters[model_adapter_name].model
        # Re-work data in sheet in order to fulfill foreignKey constraints (replace strings with objects)
        msg = "Checking ForeignKeys for '" + model_adapter_name + "'..."
        throw_or_enqueue("info", msg, main_msg_queue)
        new_sheet = setup_foreign_keys_in_sheet(sheet, model)
        if new_sheet:
            throw_or_enqueue("info", "...done.", main_msg_queue)
            data_collector_subset[model_adapter_name] = new_sheet.get_internal_array()
        else:
            throw_or_enqueue("error", "...failed!", main_msg_queue)
            error_flag = True

    if not error_flag:
        msg = "Writing data to database..."
        throw_or_enqueue("info", msg, main_msg_queue)
        try:
            save_data(importer, data_collector_subset, file_type=DB_DJANGO)
        except BulkWriteError as bwe:
            throw_or_enqueue("error", "FAILED!", main_msg_queue)
            throw_or_enqueue("error", bwe, main_msg_queue)
            return False
        throw_or_enqueue("info", "...success!", main_msg_queue)
        return True
    else:
        return False

In [226]:
def print_fault_collector(fault_collector, row_mappers, column_mappers):
    
    report = []
    throw_or_enqueue("error", "### ISSUE REPORT ###", report)

    for sheet in fault_collector.keys():
        throw_or_enqueue("error", "\n=== Sheet '" + sheet + "' ===", report)
        for column in fault_collector[sheet].keys():
            for error in fault_collector[sheet][column].keys():
                if len(fault_collector[sheet][column][error]):
                    throw_or_enqueue("error", "\nColumn '" + column + "' - '" + error + "' errors in following rows:", report)
                    throw_or_enqueue("error", ", ".join([str(row_mappers[sheet][x] + 2) for x in fault_collector[sheet][column][error]]), report)
    for msg in report:
        print(msg)

In [ ]:
### MAIN ###

In [344]:
# 1a. Core Model: Initialization
main_msg_queue = []
sheet_names = get_book(file_name=filepath).sheet_names()
misses = [sn for sn in model2sheet_core.values() if not sn in sheet_names]
if misses:
    msg = "Could not find following required worksheets in uploaded datamodel file:\n" + "\n".join(misses)
    throw_or_enqueue("error", msg)
    
if write_mode == "new":
    throw_or_enqueue("warning", "This will delete all data model information contained in the current database.")
    drop_tables_core()

In [345]:
# 2a. Core Model: start iterative reading of contents from file
dli = 0
importer_collector = {}
data_collector = {}
fault_collector = {}
column_mappers = {}
row_mappers = {}
unimported_codes = []
error_flag = False
for models in get_dependency_levels_core():
    msg = "Checking level " + str(dli+1) + " models..."
    throw_or_enqueue("info", msg, main_msg_queue)
    importer = DjangoModelImporter()  # generate new importer (adapter queue) for current dependency level
    
    for model in models:
        sheet_name = model2sheet_core[model]
        
        msg = "    => " + sheet_name + " --> " + model.__name__
        throw_or_enqueue("info", msg, main_msg_queue)
        
        # Get original data (sheet) from file
        sheet = get_sheet(file_name=filepath,
                          sheet_name=sheet_name,
                          name_columns_by_row=header_line_core[sheet_name],
                          auto_detect_float=False,  # does not work
                          auto_detect_datetime=False)
    
        adapter, fc, rm, cm = sheet2model_core(sheet, model, data_collector, unimported_codes)
        fault_collector[sheet_name] = fc
        row_mappers[sheet_name] = rm
        column_mappers[sheet_name] = cm
        # add adapter to importer (= queue for current dependency level)
        if not adapter:
            error_flag = True            
        else:
            importer.append(adapter)
            if sheet_name == "Codes":
                unimported_codes = sheet.column["Code"]   # bulk import only
                
    # END OF LOOP over sheets (= models) of the current dependency level
    
    if error_flag:
        msg = "General errors on dependency level " + str(dli+1) + " models..."
        throw_or_enqueue("error", msg, main_msg_queue)
        error_flag = True
        break
    importer_collector[dli] = importer
    dli += 1
# END OF LOOP over model dependency levels

[]
['None', '%', 's', 'min', 'h', 'd', 'mo', 'a', 'words', 'figures', 'points', 'documents', 'U/l', '?', 'mg/dl', 'mg/l', 'ml/min', 'g/dl', 'G/l', 'mmHg', 'pg/ml', '1/µl']
{'d', 's', 'U/l', 'mg/l', 'points', 'figures', 'g/dl', 'a', 'mg/dl', '?', 'mo', 'mmHg', '%', 'h', 'min', 'G/l', '1/µl', 'None', 'ml/min', 'pg/ml', 'documents', 'words'}


In [346]:
# 3a. Write collected, model-adapted data into database
if not error_flag:
    for level in range(dli):
        # fire up the import for a dependency level
        msg = "Importing level " + str(level + 1) + " model data..."
        throw_or_enqueue("info", msg, main_msg_queue)
        importer = importer_collector[level]  # get level-specific subset of adapters (= one importer)
        success = importer2database(importer, data_collector)
        if not success:
            error_flag = True
            msg = "Failed to write datamodel contents to DB (dep. level " + str(level) + ")"
            throw_or_enqueue("error", msg, main_msg_queue)
            error_flag = True
        else:
            msg = "...done: Finished on level " + str(level + 1) + " models.\n"
            throw_or_enqueue("info", msg, main_msg_queue)
else:
    throw_or_enqueue("error", "Errors occurred...", main_msg_queue)

In [347]:
# 4. Reporting
for msg in main_msg_queue:
    print(msg)
main_msg_queue = []

if error_flag:
    print_fault_collector(fault_collector, row_mappers, column_mappers)
else:
    throw_or_enqueue("info", "No errors occured")

INFO: Checking level 1 models...
INFO:     => Sources --> DatamodelSource
INFO: 	Working on columns:
INFO: 	 * Abbreviation
INFO: 	 * Source
INFO: 	 * PID_colname
INFO: 	 * SITE_colname
INFO: 	 * TIMESTAMP_colname
INFO: 	 * Header_offset
INFO:     => Units --> DatamodelUnit
INFO: 	Working on columns:
INFO: 	 * Unit
INFO: 	 * UCUM
INFO: 	 * Description
INFO:     => Codes --> DatamodelCode
INFO: 	Working on columns:
INFO: 	 * Active
INFO: 	 * Code
INFO: 	 * Code_Description
INFO: 	 * Key
INFO: 	 * Value
INFO: Checking level 2 models...
INFO:     => Attributes --> DatamodelAttribute
INFO: 	Working on columns:
INFO: 	 * Active
INFO: 	 * Topic
INFO: 	 * Topic_Description
INFO: 	 * Umbrella
INFO: 	 * Umbrella_Description
INFO: 	 * Attribute
INFO: 	 * Attribute_Description
INFO: 	 * Attribute_Tooltip
INFO: 	 * Datatype
INFO: 	 * Domain
INFO: 	 * Unit
INFO: Importing level 1 model data...
INFO: Checking ForeignKeys for 'datamodelsource'...
INFO: ...done.
INFO: Checking ForeignKeys for 'datamod

In [687]:
# 1b. Mappings: Initialization
main_msg_queue = []
sheet_names = get_book(file_name=filepath).sheet_names()
misses = [sn for sn in model2sheet_mapping.values() if not sn in sheet_names]
if misses:
    msg = "Could not find following required worksheets in uploaded datamodel file:\n" + "\n".join(misses)
    throw_or_enqueue("error", msg)
    
if write_mode == "new":
    throw_or_enqueue("warning", "This will delete all mapping information contained in the current database.")
    drop_tables_mapping()

In [688]:
# 2b. Mappings: start iterative reading of contents from file
dli = 0
importer_collector = {}
data_collector = {}
fault_collector = {}
row_mappers = {}
column_mappers = {}
unimported_codemappings = {}
error_flag = False
for models in get_dependency_levels_mapping():
    msg = "Checking level " + str(dli+1) + " models..."
    throw_or_enqueue("info", msg, main_msg_queue)
    importer = DjangoModelImporter()  # generate new importer (adapter queue) for current dependency level
    
    for model in models:
        sheet_name = model2sheet_mapping[model]
        
        msg = "    => " + sheet_name + " --> " + model.__name__
        throw_or_enqueue("info", msg, main_msg_queue)
        
        # Get original data (sheet) from file
        sheet = get_sheet(file_name=filepath,
                          sheet_name=sheet_name,
                          name_columns_by_row=header_line_mapping[sheet_name],
                          auto_detect_float=False,  # does not work
                          auto_detect_datetime=False)
    
        adapter, fc, rm, cm = sheet2model_mapping(sheet, model, data_collector, unimported_codemappings)
        fault_collector[sheet_name] = fc
        row_mappers[sheet_name] = rm
        column_mappers[sheet_name] = cm
        # add adapter to importer (= queue for current dependency level)
        if not adapter:
            error_flag = True            
        else:
            importer.append(adapter)
            if sheet_name == "Code_Mappings":                       # for bulk import only
                pos_cm = sheet.colnames.index("Code_Mapping")
                pos_te = sheet.colnames.index("Target_Equivalent")
                for row in sheet:
                    cm = row[pos_cm]
                    if cm in unimported_codemappings:
                        unimported_codemappings[cm].add(row[pos_te])
                    else:
                        unimported_codemappings[cm] = set([row[pos_te]])
                for cm in unimported_codemappings:
                    unimported_codemappings[cm] = list(unimported_codemappings[cm])
                #unimported_codemappings = sheet.column["Code_Mapping"]
                
    # END OF LOOP over sheets (= models) of the current dependency level
    
    if error_flag:
        msg = "General errors on dependency level " + str(dli+1) + " models..."
        throw_or_enqueue("error", msg, main_msg_queue)
        break
    importer_collector[dli] = importer
    dli += 1
# END OF LOOP over model dependency levels

27: FORMULA( (SW_BNTTOT - 100) / 10) ) ==> FORMULA((SW_BNTTOT - 100) / 10))
28: FORMULA( (SW_CERCP - 100) / 10) ) ==> FORMULA((SW_CERCP - 100) / 10))
29: FORMULA( (SW_CERCPR - 100) / 10) ) ==> FORMULA((SW_CERCPR - 100) / 10))
30: FORMULA( (SW_CERCPSAV - 100) / 10) ) ==> FORMULA((SW_CERCPSAV - 100) / 10))
31: FORMULA( (SW_CERDISC - 100) / 10) ) ==> FORMULA((SW_CERDISC - 100) / 10))
32: FORMULA( (SW_CERDRLCT - 100) / 10) ) ==> FORMULA((SW_CERDRLCT - 100) / 10))
33: FORMULA( (SW_CERDRLIT - 100) / 10) ) ==> FORMULA((SW_CERDRLIT - 100) / 10))
34: FORMULA( (SW_CERRL - 100) / 10) ) ==> FORMULA((SW_CERRL - 100) / 10))
35: FORMULA( (SW_CERRL1CT - 100) / 10) ) ==> FORMULA((SW_CERRL1CT - 100) / 10))
36: FORMULA( (SW_CERRL3CT - 100) / 10) ) ==> FORMULA((SW_CERRL3CT - 100) / 10))
37: FORMULA( (SW_CERWLSAV - 100) / 10) ) ==> FORMULA((SW_CERWLSAV - 100) / 10))
38: FORMULA( (SW_MMSTOT - 100) / 10) ) ==> FORMULA((SW_MMSTOT - 100) / 10))
39: FORMULA( (SW_PFSTOT - 100) / 10) ) ==> FORMULA((SW_PFSTOT - 10

In [690]:
# 3b. Write collected, model-mapped data into database
if not error_flag:
    for level in range(dli):
        # fire up the import for a dependency level
        msg = "Importing level " + str(level + 1) + " model data..."
        throw_or_enqueue("info", msg, main_msg_queue)
        importer = importer_collector[level]  # get level-specific subset of adapters (= one importer)
        success = importer2database(importer, data_collector)
        if not success:
            error_flag = True
            msg = "Failed to write datamodel contents to DB (dep. level " + str(level) + ")"
            throw_or_enqueue("error", msg, main_msg_queue)
            error_flag = True
        else:
            msg = "...done: Finished on level " + str(level + 1) + " models.\n"
            throw_or_enqueue("info", msg, main_msg_queue)
else:
    throw_or_enqueue("error", "Errors occurred...", main_msg_queue)

^^^^^^^^^^^
{'datamodelcodemapping': [[1, 'cm001', 'm', 'male', '0', ''], [1, 'cm001', 'f', 'female', '1', ''], [1, 'cm001', 'Male', 'male', '0', ''], [1, 'cm001', 'Female', 'female', '1', ''], [1, 'cm002', '0', 'no', '0', ''], [1, 'cm002', '1', 'yes', '1', ''], [1, 'cm002a', '0', 'no', '0', ''], [1, 'cm002a', '1', 'yes', '1', ''], [1, 'cm002a', '2', 'not mentioned', '2', ''], [1, 'cm002a', '3', 'equivocal', '3', ''], [1, 'cm002a', '4', 'unknown', '2', ''], [1, 'cm003', '1', 'Innerhalb der letzten sechs Monate', '[0:6]', ''], [1, 'cm003', '2', 'Vor sechs Monaten bis zwei Jahren', '[6:24]', ''], [1, 'cm003', '3', 'Vor zwei bis fünf Jahren', '[24:60]', ''], [1, 'cm003', '4', 'Vor mehr als fünf Jahren', '[60:]', ''], [1, 'cm004', 'SCA1', 'SCA1', '1', ''], [1, 'cm004', 'SCA2', 'SCA2', '2', ''], [1, 'cm004', 'SCA3', 'SCA3', '3', ''], [1, 'cm004', 'SCA6', 'SCA6', '6', ''], [1, 'cm005', 'Jan', '', '1', ''], [1, 'cm005', 'Feb', '', '2', ''], [1, 'cm005', 'Mar', '', '3', ''], [1, 'cm005', 'Apr'

^^^^^^^^^^^
{'datamodelcalculation': [[1, 0, <DatamodelSource: DatamodelSource object (CRCSCA)>, 'SFINGERMEAN_CALC', 'MEAN(SCHASERI, SCHASELE, 2d)', ''], [1, 0, <DatamodelSource: DatamodelSource object (CRCSCA)>, 'SFINGERMEAN', 'RANK(SFINGERMEAN_CALC, SFINGERMEAN, 2d)', ''], [1, 0, <DatamodelSource: DatamodelSource object (CRCSCA)>, 'SFINGERNOSEMEAN_CALC', 'MEAN(SFINGERRI, SFINGERLE, 2d)', ''], [1, 0, <DatamodelSource: DatamodelSource object (CRCSCA)>, 'SFINGERNOSEMEAN', 'RANK(SFINGERNOSEMEAN_CALC, SFINGERNOSEMEAN, 2d)', ''], [1, 0, <DatamodelSource: DatamodelSource object (CRCSCA)>, 'SALTHANDMEAN_CALC', 'MEAN(ALHANDRI, ALHANDLE, 2d)', ''], [1, 0, <DatamodelSource: DatamodelSource object (CRCSCA)>, 'SALTHANDMEAN', 'RANK(SALTHANDMEAN_CALC, SALTHANDMEAN, 2d)', ''], [1, 0, <DatamodelSource: DatamodelSource object (CRCSCA)>, 'SHEELSHINMEAN_CALC', 'MEAN(SHEELRI, SHEELLE, 2d)', ''], [1, 0, <DatamodelSource: DatamodelSource object (CRCSCA)>, 'SHEELSHINMEAN', 'RANK(SHEELSHINMEAN_CALC, SHEELSHI

In [691]:
# 4. Reporting
for msg in main_msg_queue:
    print(msg)
main_msg_queue = []

if error_flag:
    print("=== E R R O R S ===")
    print_fault_collector(fault_collector, row_mappers, column_mappers)
else:
    print("= no errors =")
    throw_or_enqueue("info", "No errors occured")

INFO: Checking level 1 models...
INFO:     => Code_Mappings --> DatamodelCodeMapping
INFO: 	Working on columns:
INFO: 	 * Active
INFO: 	 * Code_Mapping
INFO: 	 * Source_Value
INFO: 	 * Source_Value_Description
INFO: 	 * Target_Equivalent
INFO: 	 * Remarks
INFO: Checking level 2 models...
INFO:     => Attribute_Mappings --> DatamodelAttributeMapping
INFO: 	Working on columns:
INFO: 	 * Active
INFO: 	 * Source
INFO: 	 * Source_Attribute
INFO: 	 * Target_Attribute
INFO: 	 * Transformation
INFO: Checking level 3 models...
INFO:     => Calculations --> DatamodelCalculation
INFO: 	Working on columns:
INFO: 	 * Active
INFO: 	 * Workbench
INFO: 	 * Source
INFO: 	 * Attribute
INFO: 	 * Function
INFO: 	 * Remarks
INFO: Importing level 1 model data...
INFO: Checking ForeignKeys for 'datamodelcodemapping'...
INFO: ...done.
INFO: Writing data to database...
INFO: ...success!
INFO: ...done: Finished on level 1 models.

INFO: Importing level 2 model data...
INFO: Checking ForeignKeys for 'datamodelat

In [ ]:
### END OF MAIN ###

In [ ]:
### Load Core ###

# 0. setup message queue
main_msg_queue = []

# 1. generate sheet object from input data (form => array => pyexcel sheet/book)
###

# 2. test sheet data vs. selected model; unimported_codes are those from a 'Codes' sheet, if not yet sent to DB
data_collector = []
unimported_codes = []   # or fill with contents
adapter, fault_collector, row_mapper, column_mapper = sheet2model_core(sheet, model, data_collector, unimported_codes)
# fault_collector should (structure, but no data); adapter might by False, if errors occured

# 3. enqueue adapter into importer and write to DB
importer = DjangoModelImporter()
success = importer2database(importer, data_collector)

# 4. read log and report errors
for msg in main_msg_queue:
    print(msg)
main_msg_queue = []
read_fault_collector(fault_collector, row_mapper, column_mapper)

In [ ]:
### Load Mappings ###

# 0. setup message queue
main_msg_queue = []

# 1. generate sheet object from input data (form => array => pyexcel sheet/book)
###

# 2. test sheet data vs. selected model; unimported_codes are those from a 'Codes' sheet, if not yet sent to DB
data_collector = []
unimported_codes = []   # or fill with contents
adapter, fault_collector, row_mapper, column_mapper = sheet2model_mapping(sheet, model, data_collector, unimported_codes)
# fault_collector should (structure, but no data); adapter might by False, if errors occured

# 3. enqueue adapter into importer and write to DB
importer = DjangoModelImporter()
success = importer2database(importer, data_collector)

# 4. read log and report errors
for msg in main_msg_queue:
    print(msg)
main_msg_queue = []
read_fault_collector(fault_collector, row_mapper, column_mapper)

In [52]:
sheet_names

['Sources',
 'Units',
 'Attributes',
 'Codes',
 'Attribute_Mappings',
 'Code_Mappings',
 'Calculations',
 'Questions+Remarks']